# Mini-projet : Classification de tumeurs gliales

Les _gliomes_ ou _tumeurs gliales_ sont des tumeurs du glie, le tissu de soutien neuronal du cerveau. Elles sont classifiées en 4 grades anatomo-pathologiques, dont dépend la prise en charge.

Dans ce jeu de données, chaque observation est un gliome, décrit par l'expression de 4 434 gènes. L'expression d'un gène est une mesure de la quantité d'ARN correspondant à ce gène qui est présente dans la cellule. Schématiquement, l'ADN est transcrit en ARN, lequel est lui-même traduit en une protéine. Les protéines assurent une multitude de fonctions du vivant, mais mesurer leur quantité est difficile ; d'où l'intérêt d'utiliser les quantités d'ARN, bien que la correspondance ne soit pas immédiate. 

Chaque gliome de notre jeu de données est étiquetée en fonction de son grade. 

Le but de ce projet est de construire un classifieur qui détermine, sur la base de l'expression de ces 4 434 gènes, le grade d'un gliome.

## Instructions
1. Comparez les performances d'au moins deux algorithmes d'apprentissage sur ce problème de classification.

__Attention :__
* au _data leakage_ (ne pas utiliser les données sur lesquelles on évalue les modèles pour les entraîner ou prétraiter les données)
* à la taille du jeu de données
* au nombre de classes
* à choisir une mesure de performance appropriée (justifiez votre choix)
2. Identifiez, quand cela est possible, les gènes les plus importants pour les modèles que vous avez entraînés. S'agit-il des mêmes gènes
* entre deux modèles obtenus grâce à un algorithme d'apprentissage différents ?
* entre deux modèles obtenus en utilisant le même algorithme d'apprentissage sur des sous-échantillons différents des données ?

N'oubliez pas de commenter et interpréter vos résultats.

## Chargement des données

In [1]:
import scipy.io
data_matrix = scipy.io.loadmat("gliome.mat")

In [2]:
X = data_matrix['X']
print(X.shape)

(50, 4434)


In [3]:
y = data_matrix['Y'][:, 0] 
print(y.shape)
random_state = 42

(50,)


### D'abord, nous divisons les données en test et train.
Même si nous n'avons pas beaucoup de données, nous devons faire la division...

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV

X_std = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=random_state)

# Métrique
Nous utiliserions normalement la métrique "précision" car nos données sont bien stratifiées, c'est-à-dire que nous avons la même quantité de classes pour chaque classe y_i.
Cependant, nous pouvons penser que la précision de la classe 4 est plus importante que la précision de la classe 1, car lorsque nous identifions la classe 4, nous avons besoin de faire une opération et nous ne voulons pas faire des opérations sans être nécessaire. D'autre part, le rappel de la classe 1 est également très important, car si le gliome est découvert à un stade précoce, il peut être traité et sa détérioration peut être évitée. Nous pourrions donc utiliser la métrique (Précision_4 + Rappel_1) / 2.
* Obs, je n'ai pas de connaissances médicales mais les idées présentées ici peuvent être facilement modifiées en ayant un spécialiste médical dans l'équipe.
# Choix du modèle
Nous allons choisir entre la régression logistique (pour être le modèle le plus simple), le SVM (parce que nous n'avons pas beaucoup de labels et d'exemples), la forêt aléatoire (car p est élevé) et le réseau de neurones en couches (pour être le modèle le plus complexe). Nous pourrions également tester le boosting, mais nous avons déjà beaucoup d'hyperparamètres à tester.
## Hyperparamètres
Nous pouvons penser que le choix du modèle lui-même est un hyperparamètre, cependant, nous devons optimiser chaque modèle pour obtenir une comparaison équitable entre chaque modèle.
Pour ce faire, nous allons utiliser la validation croisée stratifiée (c'est-à-dire avec le même nombre de classes pour chaque division) avec 5 divisions (ainsi, nous entraînons le modèle avec 32 exemples - 4*(0,8 * 50)/5 - et le testons dans les 8 exemples restants). Ainsi, notre ensemble de données de "validation" est constitué de fractions de l'ensemble de données de "training".
### Itérations
Ici, seule l'itération "la plus fine" des hyperparamètres sera montrée. Par exemple, pour C dans MLP nous commençons avec C à [0.1,1,10,100] et ensuite [0.001,.0.01,.01] parce que les meilleurs résultats pour MLP étaient avec C = 0.1 dans l'itération précédente.

In [18]:
from collections import defaultdict
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict, StratifiedKFold
from math import sqrt

models = []
for penalty in ["l2"]:
    for solver in ["newton-cg","lbfgs"]:
        for C in [0.001,0.01,0.1]:
            if penalty == "l1" and solver in ["newton-cg","lbfgs","sag"]: # not supported
                continue
            else:
                models.append(((f'LR-{penalty}-{solver}-{C}'),LogisticRegression(penalty=penalty,C=C,max_iter=1000,solver=solver, random_state=random_state,n_jobs=10)))

for kernel in ["linear","rbf"]:
    for C in [0.1,0.5,1,5,10]:
        models.append(((f'SVM-{kernel}-{C}'),SVC(kernel=kernel,C=C,max_iter=1000, random_state=random_state)))

for n_estimators in [50,100,200,500]:
    for max_depth in [2,8,20]:
        models.append(((f'RF-{n_estimators}-{max_depth}'),RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth, random_state=random_state,n_jobs=10)))

for hidden_layer_sizes in [(300,),(400,),(150,)]:
    for activation in ["logistic"]:
        for alpha in [0.1,0.5,1,4]:
            models.append(((f'MLP-{hidden_layer_sizes}-{activation}-{alpha}'),MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,activation=activation,alpha=alpha, batch_size=3, random_state=random_state,early_stopping=True)))

def eval_models(models,n_classes = 4):
    from sklearn.base import clone
    results = {}

    for name, model in models:
        # K-fold stratifié manuellement pour plus de contrôle et pour pouvoir faire toutes les métriques
        accuracy_list = []
        model_fit_list = [] # Pour chaque fold
        class_results = defaultdict(lambda: defaultdict(list))
        kf = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True).split(X_train, y_train)
        for train_index, test_index in kf:

            X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
            y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

            model.fit(X_train_fold, y_train_fold)
            predict = model.predict(X_test_fold)
            # Fonction facile pour faire la précision et le rappel (et f1) de chaque classe.
            class_report = classification_report(y_test_fold, predict,zero_division=0,output_dict=True)
            for i in range(n_classes):
                class_results[i+1]['precision'].append(class_report[str(i+1)]['precision'])
                class_results[i+1]['recall'].append(class_report[str(i+1)]['recall'])
                class_results[i+1]['f1-score'].append(class_report[str(i+1)]['f1-score'])
                class_results[i+1]['support'].append(class_report[str(i+1)]['support'])
            accuracy_list.append(class_report['accuracy'])
            model_fit_list.append(clone(model))

        custom_metric_array = (np.array(class_results[4]['precision'] )+ np.array(class_results[1]['recall']))/2
        custom_metric = np.mean(custom_metric_array)
        custom_metric_std = np.std(custom_metric_array)
        for i in range(1,n_classes+1):
            class_results[i]['precision'] = sum(class_results[i]['precision']) / len(class_results[i]['precision'])
            class_results[i]['recall'] = sum(class_results[i]['recall']) / len(class_results[i]['recall'])
            class_results[i]['f1-score'] = sum(class_results[i]['f1-score']) / len(class_results[i]['f1-score'])

        total_accuracy = sum(accuracy_list) / len(accuracy_list)
        std_accuracy = np.std(accuracy_list)

        results[name] = {"class_results":class_results, "accuracy":total_accuracy, "std_accuracy":std_accuracy, "custom_metric":custom_metric,"custom_metric_std":custom_metric_std, "models":model_fit_list}
        print('%s: %f (%f) - %f (%f)' % (name, total_accuracy, sqrt(std_accuracy),custom_metric,custom_metric_std))
    return results
    
results = eval_models(models)


LR-l2-newton-cg-0.001: 0.775000 (0.407224) - 0.866667 (0.124722)
LR-l2-newton-cg-0.01: 0.800000 (0.411775) - 0.866667 (0.124722)
LR-l2-newton-cg-0.1: 0.800000 (0.411775) - 0.866667 (0.124722)
LR-l2-lbfgs-0.001: 0.775000 (0.407224) - 0.866667 (0.124722)
LR-l2-lbfgs-0.01: 0.800000 (0.411775) - 0.866667 (0.124722)
LR-l2-lbfgs-0.1: 0.800000 (0.411775) - 0.866667 (0.124722)
SVM-linear-0.1: 0.800000 (0.411775) - 0.866667 (0.124722)
SVM-linear-0.5: 0.800000 (0.411775) - 0.866667 (0.124722)
SVM-linear-1: 0.800000 (0.411775) - 0.866667 (0.124722)
SVM-linear-5: 0.800000 (0.411775) - 0.866667 (0.124722)
SVM-linear-10: 0.800000 (0.411775) - 0.866667 (0.124722)
SVM-rbf-0.1: 0.450000 (0.411775) - 0.606667 (0.117662)
SVM-rbf-0.5: 0.600000 (0.223607) - 0.726667 (0.099219)
SVM-rbf-1: 0.650000 (0.349964) - 0.708333 (0.134371)
SVM-rbf-5: 0.775000 (0.407224) - 0.866667 (0.124722)
SVM-rbf-10: 0.775000 (0.407224) - 0.866667 (0.124722)
RF-50-2: 0.725000 (0.349964) - 0.808333 (0.128019)
RF-50-8: 0.725000 (0.3

In [21]:
filtered_results_dict = {}
for name,result in results.items():
    if result['custom_metric'] > 0.7:
        filtered_results_dict[name] = {"custom_metric":result['custom_metric'],"custom_metric_std":result['custom_metric_std'],"accuracy":result['accuracy'],"std_accuracy":result['std_accuracy'],"model":result['models']}

In [22]:
import pickle as pkl

with open('results.pkl', 'wb') as f:
    pkl.dump(filtered_results_dict, f)

In [23]:
with open('results.pkl', 'rb') as f:
    filtered_results_dict = pkl.load(f)

In [24]:
import pandas as pd
df_models = pd.DataFrame.from_dict(filtered_results_dict,orient='index')
df_models
df_models.sort_values(by=['custom_metric',"custom_metric_std"],ascending=[False,True],inplace=True)

In [ ]:
Maintenant, on cherche les modéles avec la plus grand "custom_metric" et la plus petite "custom_metric_std"

In [25]:
df_models[:50]

,custom_metric,custom_metric_std,accuracy,std_accuracy,model
LR-l2-newton-cg-0.001,0.866667,0.124722,0.775,0.165831,"[LogisticRegression(C=0.001, max_iter=1000, n_..."
LR-l2-newton-cg-0.01,0.866667,0.124722,0.800,0.169558,"[LogisticRegression(C=0.01, max_iter=1000, n_j..."
LR-l2-newton-cg-0.1,0.866667,0.124722,0.800,0.169558,"[LogisticRegression(C=0.1, max_iter=1000, n_jo..."
LR-l2-lbfgs-0.001,0.866667,0.124722,0.775,0.165831,"[LogisticRegression(C=0.001, max_iter=1000, n_..."
LR-l2-lbfgs-0.01,0.866667,0.124722,0.800,0.169558,"[LogisticRegression(C=0.01, max_iter=1000, n_j..."
LR-l2-lbfgs-0.1,0.866667,0.124722,0.800,0.169558,"[LogisticRegression(C=0.1, max_iter=1000, n_jo..."
SVM-linear-0.1,0.866667,0.124722,0.800,0.169558,"[SVC(C=0.1, kernel='linear', max_iter=1000, ra..."
SVM-linear-0.5,0.866667,0.124722,0.800,0.169558,"[SVC(C=0.5, kernel='linear', max_iter=1000, ra..."
SVM-linear-1,0.866667,0.124722,0.800,0.169558,"[SVC(C=1, kernel='linear', max_iter=1000, rand..."
SVM-linear-5,0.866667,0.124722,0.800,0.169558,"[SVC(C=5, kernel='linear', max_iter=1000, rand..."


In [ ]:
#Verifica a coef dos modelos escolhidos ali, só usar o "models" que já tem um fit diferente pra cada fold (pergunta 1 parte 2) e compara os coef. 
# Pra pergunta 1 parte 2 só pegar o modelo (qualquer um da lista de modelos) dar fit no train inteiro e comparar modelos diferentes (e a eficacia deles no test)

SVM importance: https://stackoverflow.com/questions/41592661/determining-the-most-contributing-features-for-svm-classifier-in-sklearn
random forest importance: https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html
mlp: on peut pas
logistic_regression: utiliser "coef_"
